In [102]:
#Import dependencies
import pandas as pd

#Read in the csv file
datapath = "Resources/purchase_data.csv"
rawdata = pd.read_csv(datapath)



### purchase_data.CSV Data Sample

In [154]:
#Display data sample
rawdata.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


---

# Player Count
Find the number of unique entries in the SN column of the data

In [155]:
playercount = (
    rawdata.agg({'SN':'nunique'})
    .to_frame().rename({'SN': 'Total Players'}).T # Improve the format of the output
)

Display the data

In [104]:
display(playercount)

,Total Players
0,576


---

# Purchasing Analysis (Total)

Obtain a fresh copy of the data with entries for all transactions

In [157]:
purchasing_analysis = rawdata.copy()
purchasing_analysis["Total Revenue"] = purchasing_analysis["Price"]

Get aggregate statistical measures from all transactions

In [158]:
purchasing_analysis = (
    purchasing_analysis
    .agg({"Purchase ID":'count',"Price":'mean',
          "Item ID":'nunique',"Total Revenue":'sum'})
    .rename({"Purchase ID":"Number of Purchases","Price":"Average Price", # Improve the format of the output
             "Item ID":"Number of Unique Items"}).to_frame().T
)

Format and display the data

In [159]:
purchasing_analysis.head().style.format({"Number of Purchases":int,"Average Price":"${:.2f}",
                                         "Number of Unique Items":int,"Total Revenue":'${:.2f}'})

,Number of Purchases,Average Price,Number of Unique Items,Total Revenue
0,780,$3.05,179,$2379.77


---

# Gender Demographics

Obtain a fresh copy of the data with a single entry for each player

In [145]:
genderdemos = rawdata.copy().drop_duplicates(subset = "SN",ignore_index =True)

Group the entries by gender, and count the results

In [145]:
genderdemos=genderdemos["Gender"].value_counts().to_frame().rename(columns = {"Gender":"Number of Players"})
genderdemos["Percentage of Players"]= genderdemos["Number of Players"]/sum(genderdemos["Number of Players"])

Format and display the data

In [145]:
genderdemos.head().style.format({"Percentage of Players": "{:.2%}"})

,Number of Players,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


 ---
 
 # Purchasing Analysis (Gender)

Get aggregate statistical measures from data containing all transactions

In [107]:
genderpurchasing =(
    rawdata.groupby("Gender").agg({"Purchase ID":"count","Price":["mean","sum"]})
    .droplevel(0,1)
    .rename(columns ={"count":"Total Purchases","mean":"Average Price","sum":"Total Revenue"})
)

Group the data to get total sales for each player, then return the mean total purchase for each gender

In [107]:
genderpurchasing["Average Per Person"] = (rawdata.groupby("SN").agg({"Gender":'first','Price': 'sum'})
                                          .groupby("Gender").mean())

Format and display the data

In [146]:
genderpurchasing.head().style.format({"Average Price":'${:.2f}',"Total Revenue":"${:.2f}",
                                      "Average Per Person":"${:.2f}"})

,Total Purchases,Average Price,Total Revenue,Average Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


---

# Age Demographics 

Set up bin sizes and labels for pd.cut()

In [134]:
agebins = [0,10,15,20,25,30,35,40,125]
agebins_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39",">40"]

Get a fresh data copy with a single age entry for each player, then bin the ages into groups

In [134]:
agedata = rawdata.copy().drop_duplicates(subset = "SN",ignore_index =True)
binned_ages_df = pd.cut(agedata["Age"],agebins,labels = agebins_names, right =False)

Count the results

In [153]:
agedemographics = (binned_ages_df.value_counts().to_frame().reindex(agebins_names)
                   .rename(columns={"Age": "Number of Players"})) # Change column names to reflect new meaning
agedemographics["Percent of Players"] = agedemographics["Number of Players"]/sum(agedemographics["Number of Players"])

Format and display the data

In [152]:
agedemographics.head(8).style.format({"Percent of Players":"{:.2%}"})

,Number of Players,Percent of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
>40,12,2.08%


---

# Purchasing Analysis (Age) 

Get a fresh copy of the data that has entries for all transaction, then bin the ages into groups

In [149]:
agegroups = rawdata.copy()
agegroups["Age Range"]= pd.cut(rawdata["Age"],agebins,labels = agebins_names,ordered = True,right= False) 

Get aggregate statistical measures for each age group

In [148]:
ageanalysis = (
    agegroups.groupby("Age Range").agg({"Purchase ID":'count',"Price":['mean','sum']})
    .droplevel(0,1).rename(columns ={"count":"Total Purchases","mean": 
                                     "Average Price", "sum":"Total Revenue"}) #Improve the column headers
)
ageanalysis["Average Per Person"] = (
    agegroups.copy().groupby("SN").agg({"Age Range":'first','Price': 'sum'})
    .groupby("Age Range").agg({"Price":'mean'})
)

Format and display the data

In [143]:
ageanalysis.head(8).style.format({"Average Price":"${:.2f}",
                                  "Total Revenue":"${:.2f}",
                                  "Average Per Person":"${:.2f}"})

,Total Purchases,Average Price,Total Revenue,Average Per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
>40,13,$2.94,$38.24,$3.19


---

# Top Spenders

In [92]:
topspenders = (
    rawdata.copy().groupby("SN").agg({"Purchase ID":'count',"Price":['mean','sum']})
    .droplevel(0,1)
    .rename(columns = {"count":"Purchase Count","mean":"Price","sum":"Total Revenue"})
    .sort_values("Total Revenue",ascending = False)
)
topspenders.head().style.format({"Price":"${:.2f}","Total Revenue":"${:.2f}"})

,Purchase Count,Price,Total Revenue
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


---

# Most Popular Items

In [160]:
popularitems = (
    rawdata.copy().groupby(["Item ID","Item Name"]).agg({"Purchase ID":'count',"Price":['mean','sum']})
    .droplevel(0,1)
    .rename(columns = {"count":"Purchase Count","mean":"Price","sum":"Total Revenue"})
    .sort_values("Purchase Count",ascending = False)
)
popularitems.head().style.format({"Price":"${:.2f}","Total Revenue":"${:.2f}"})

,,Purchase Count,Price,Total Revenue
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


---

# Most Profitable Items 

Sort the most popular items table to determine te most profitable items

In [94]:
profitable_items = popularitems.sort_values("Total Revenue",ascending = False)

Format and display the data

In [94]:
profitable_items.head().style.format({"Price":"${:.2f}","Total Revenue":"${:.2f}"})

,,Purchase Count,Price,Total Revenue
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
